In [ ]:
import tabula
import pandas as pd
import os

In [ ]:
# Nombre del archivo PDF a procesar
# Carpeta de origen donde se encuentran los archivos PDF
carpeta_origen = r'D:\DOCUMENTOS\Resumen Bancarios\Resumenes PDF'
# Carpeta de destino donde se guardarán los archivos procesados
carpeta_destino = r'D:\DOCUMENTOS\Resumen Bancarios\Resumen Excel'
# Nombres de las columnas deseadas para el DataFrame final
columnas = ["Fecha", "Descripcion", "Referencia", "Debitos", "Creditos", "Saldos"]

In [ ]:
def procesar_pdf(archivo, carpeta_destino):
    """
    Procesa un archivo PDF de extracto bancario y guarda los datos extraídos en un archivo CSV.

    Args:
        archivo (str): Ruta del archivo PDF a procesar.
        carpeta_destino (str): Carpeta donde se guardará el archivo CSV procesado.
    """
    # EXTRACCIÓN DEL NÚMERO DE CUENTA Y PERIODO
    df_cta = tabula.read_pdf(
        archivo,          # Ruta del archivo PDF a procesar
        pages=1,          # Página específica del PDF que queremos leer (en este caso, la primera página)
        relative_area=True, # Indica que las coordenadas del área están en porcentaje en lugar de puntos
        relative_columns=True, # Indica que las coordenadas de las columnas están en porcentaje en lugar de puntos
        area=[15, 0, 35, 100], # Área en la que buscar los datos, especificada como [top, left, bottom, right] en porcentajes
        columns=[80],     # Columnas específicas dentro del área seleccionada, especificadas en porcentaje
        pandas_options={'header': None} # Opciones adicionales para leer el PDF en un DataFrame de pandas, en este caso, indicando que no hay encabezado
    )

    df_cbu = df_cta[0]
    Nro_Cuenta = df_cbu.iloc[0, 0][-18:]  # Extrae los últimos 18 caracteres como el número de cuenta
    Periodo = df_cbu.iloc[4, 0][21:].strip()  # Extrae y limpia el periodo
    Saldo_inicial = float(df_cbu.iloc[3, 0].split(" ")[0].replace(',', ''))  # Convierte el saldo inicial a float

    # Divide el periodo en fechas de inicio y fin
    desde, hasta = Periodo.split(" al ")
    fecha_desde = "-".join(reversed(desde.split("/")))
    fecha_hasta = "-".join(reversed(hasta.split("/")))

    # Formatea las fechas en el formato deseado para el nombre del archivo
    date_name = f"{fecha_desde} al {fecha_hasta}"
    name_archivo = Nro_Cuenta + " Periodo " + date_name + '.csv'

    # EXTRACCIÓN DE LOS MOVIMIENTOS BANCARIOS
    df_pdf = tabula.read_pdf(
        archivo, pages='all',
        relative_area=True, 
        relative_columns=True,
        area=[36, 0, 90, 100],
        columns=[11, 50, 60, 70, 85], 
        pandas_options={'header': None}
    )

    # Renombrar las columnas de los DataFrames extraídos
    df_renombrados = [df.rename(columns=dict(zip(df.columns, columnas))) for df in df_pdf]
    # Unir todos los DataFrames en uno solo
    df_unido = pd.concat(df_renombrados, ignore_index=True)

    # Eliminar espacios en blanco en las descripciones
    df_unido['Descripcion'] = df_unido['Descripcion'].str.strip()

    # Filtrar filas innecesarias
    df_filtrado = df_unido[~df_unido['Descripcion'].str.contains('SALDO FINAL DEL DIA|DESCRIPCION', case=False, na=False)]
    
    if len(df_filtrado) > 1:
        # Filtrar filas vacías
        df_filtrado = df_filtrado[~df_filtrado['Fecha'].isna()]

        # Eliminar la columna 'Saldos'
        df_filtrado = df_filtrado.drop(columns=['Saldos'])

        # Reemplazar None por '0' en las columnas 'Débitos' y 'Créditos'
        df_filtrado['Debitos'] = df_filtrado['Debitos'].fillna('0')
        df_filtrado['Creditos'] = df_filtrado['Creditos'].fillna('0')

        # Eliminar comas de separador de miles y convertir a float
        df_filtrado['Debitos'] = df_filtrado['Debitos'].str.replace(',', '').astype(float)
        df_filtrado['Creditos'] = df_filtrado['Creditos'].str.replace(',', '').astype(float)

        # Calcular la columna "Saldo"
        df_filtrado['Saldo'] = Saldo_inicial + (df_filtrado['Creditos'].cumsum() - df_filtrado['Debitos'].cumsum()).round(2)

        # Renombrar las columnas
        df_filtrado.columns = ['Fecha', 'Descripcion', 'Referencia', 'Debitos', 'Creditos', 'Saldo']
    
    # Guardar el DataFrame procesado en un archivo CSV
    ubicacion_excel = os.path.join(carpeta_destino, name_archivo)
    df_filtrado.to_csv(ubicacion_excel, sep=';', encoding='latin-1',  decimal=',', float_format='%.2f', index=False)

In [ ]:
# Lista de archivos PDF en la carpeta de origen
Lista_archivos = [f for f in os.listdir(carpeta_origen) if f.endswith('.pdf')]

# Procesar cada archivo PDF en la lista
for archivo in Lista_archivos:
    pdf_archivo = os.path.join(carpeta_origen, archivo)
    try:
        procesar_pdf(pdf_archivo, carpeta_destino)
        print(f"Archivo procesado correctamente: {archivo}")
    except Exception as e:
        print(f"Error en el archivo: {archivo}")
        print(f"Detalles del error: {e}")
